# Language Detection in MP3s using Librosa and Keras
* Noah Sagrow
* David Haase

## Import Libraries

In [1]:
import os
import re

import pandas as pd
import numpy as np

from pydub import AudioSegment

from audioutils import AudioSplitter

target_dir = '/Volumes/LaCie Rose Gold 2TB/Datasets/'

## Load Data
* Audio files split into 10-second extracts
* 5,256 extracts created

### Extract Features

In [2]:
seconds = 10
res_type = 'kaiser_fast'
Audio_10 = AudioSplitter(target_dir=target_dir, test_split=seconds)